In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import sys
sys.path.insert(0, 'drive/Graduation Project/Colaboratory')

In [5]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11332833682080485363, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11288962663
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3074035257072341949
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [0]:
df = pd.read_csv('drive/Graduation Project/Colaboratory/user_data_anonymized.csv')
df = df.drop([df.columns[0]],axis=1)

In [9]:
df.head()

,user_id,username,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,type,ntrial,timestamp
0,0,User_1,0.032867,-0.011398,-0.989441,0.001340,-0.004726,0.000653,0,3,2018-03-15 21:49:02.853
1,0,User_1,0.036270,-0.646866,-0.749863,0.001340,-0.004726,0.000653,0,3,2018-03-15 21:49:02.876
2,0,User_1,0.036270,-0.646866,-0.749863,0.001340,-0.004726,0.000653,0,3,2018-03-15 21:49:02.903
3,0,User_1,-0.061005,-0.691040,-0.748917,0.320133,0.039149,-0.234852,0,3,2018-03-15 21:49:03.052
4,0,User_1,-0.140427,-0.746765,-0.741333,0.381414,0.046364,-0.343133,0,3,2018-03-15 21:49:03.074


In [0]:
cols = ['sensor_type','trial','name','type','set']
SEQ_LENGTH = 303
OVERLAP = 0
for i in range(SEQ_LENGTH):
    cols.append('x'+str(i))
moving_window_x = pd.DataFrame(columns = cols)

label = pd.DataFrame(columns = ['sensor_type','set','type1','type2','type3','type4'])
s_index = 0
for trial in np.sort(df['ntrial'].unique()):
    nt = df.loc[df['ntrial'] == trial]

    names = df['username'].unique()
    for name in names:
        n = nt.loc[nt['username'] == name]
        
        types = np.array([1,2,3,4])
        for type_ in types:
            t = n.loc[n['type'] == type_]
            
            sensors = t.columns[2:8]
            for sensor in sensors:  
                s = t
                
                if trial == 5: text="test"
                else: text="train"
                    
                row = np.array([sensor])
                row = np.append(row,[trial])
                row = np.append(row,[name])
                row = np.append(row,[type_])
                row = np.append(row,[text])
                if s[sensor].values.shape[0] > SEQ_LENGTH:
                    row = np.append(row,s[sensor].values[:SEQ_LENGTH].astype(float))
                    #print(s[sensor].values[:SEQ_LENGTH])
                elif s[sensor].values.shape[0] <= SEQ_LENGTH:
                    padded_row = np.append(s[sensor].values[:].astype(float),np.array([s[sensor].values[-1].astype(float) for i in range(SEQ_LENGTH-s[sensor].values.shape[0])]))
                    row = np.append(row,padded_row)
                moving_window_x = moving_window_x.append(pd.DataFrame([row],columns=moving_window_x.columns))
                
                row_label = np.array([sensor])
                row_label = np.append(row_label,[text])
                row_label = np.append(row_label,np.zeros((types.shape[0],)))
                row_label[type_+1] = 1.0
                label = label.append(pd.DataFrame([row_label],columns=label.columns))
                
    s_index += s[sensor].shape[0]

In [0]:
moving_window_x['index'] = [i for i in range(moving_window_x.shape[0])]
moving_window_x = moving_window_x.set_index(['index'])

label['index'] = [i for i in range(label.shape[0])]
label = label.set_index(['index'])

In [0]:
train_len = int(moving_window_x.loc[moving_window_x['set'] == 'train'].shape[0]/6)
test_len = int(moving_window_x.loc[moving_window_x['set'] == 'test'].shape[0]/6)

In [0]:
x_train = np.zeros((train_len,SEQ_LENGTH,6))
y_train = np.zeros((train_len,4))

x_test = np.zeros((test_len,SEQ_LENGTH,6))
y_test = np.zeros((test_len,4))

In [0]:
shuffle_data = np.zeros((train_len+test_len,SEQ_LENGTH,6+4))

In [15]:
for i,sensor in enumerate(moving_window_x['sensor_type'].unique()):
    s_train = moving_window_x.loc[moving_window_x['sensor_type'] == sensor]
    s_train = s_train.loc[s_train['set'] == "train"]
    s_test = moving_window_x.loc[moving_window_x['sensor_type'] == sensor]
    s_test = s_test.loc[s_test['set'] == "test"]

    x_train[:,:,i] = s_train.drop(['sensor_type','trial','name','type','set'],axis=1)
    x_test[:,:,i] = s_test.drop(['sensor_type','trial','name','type','set'],axis=1)
    if i == 0:
        l = label.loc[label['sensor_type'] == sensor]
        l_train = l.loc[l['set'] == "train"]
        l_test = l.loc[l['set'] == "test"]
        
        y_train[:,:] = l_train.drop(['sensor_type','set'],axis=1)
        y_test[:,:] = l_test.drop(['sensor_type','set'],axis=1)

print("x_train.shape: "+str(x_train.shape))
print("y_train.shape: "+str(y_train.shape))
print("x_test.shape: "+str(x_test.shape))
print("y_test.shape: "+str(y_test.shape))

x_train.shape: (32, 303, 6)
y_train.shape: (32, 4)
x_test.shape: (8, 303, 6)
y_test.shape: (8, 4)


In [16]:
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM
from keras.callbacks import History
import seaborn as sn
import time

Using TensorFlow backend.


In [17]:
y_train_ = np.zeros((32,303,4))
y_train_[:,0,:] = y_train

y_test_ = np.zeros((8,303,4))
y_test_[:,0,:] = y_test

shuffle_data[:32,:,:6] = x_train
shuffle_data[:32,:,6:] = y_train_

shuffle_data[32:,:,:6] = x_test
shuffle_data[32:,:,6:] = y_test_


while True:
  np.random.shuffle(shuffle_data)

  x_train = shuffle_data[:32,:,:6]
  x_test = shuffle_data[32:,:,:6]
  y_train = shuffle_data[:32,0,6:].reshape(32,4)
  y_test = shuffle_data[32:,0,6:].reshape(8,4)
  if np.sum(y_test[:,0]) == 2:
      if np.sum(y_test[:,1]) == 2:
          if np.sum(y_test[:,2]) == 2:
              if np.sum(y_test[:,3]) == 2:
                  break


batch_size = 1
timesteps = SEQ_LENGTH
data_dim = 6

model = Sequential()
model.add(CuDNNLSTM(128, return_sequences=True, 
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(CuDNNLSTM(64))
model.add(Dense(32))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


st = time.time()
history = model.fit(x_train, y_train,
          batch_size=batch_size, epochs=100, shuffle=False,
          validation_data=(x_test, y_test),verbose=1)
ft = time.time()

print("batch size : %d -> %d seconds for training with GPU"%(batch_size,ft-st))


Train on 32 samples, validate on 8 samples
Epoch 1/100
32/32 [==============================] - 4s 114ms/step - loss: 1.4876 - acc: 0.1875 - val_loss: 1.4368 - val_acc: 0.1250
Epoch 2/100
32/32 [==============================] - 2s 74ms/step - loss: 1.3481 - acc: 0.4062 - val_loss: 1.4721 - val_acc: 0.1250
Epoch 3/100
32/32 [==============================] - 2s 74ms/step - loss: 1.3670 - acc: 0.4062 - val_loss: 1.4558 - val_acc: 0.1250
Epoch 4/100
32/32 [==============================] - 2s 75ms/step - loss: 1.2849 - acc: 0.4062 - val_loss: 1.3615 - val_acc: 0.2500
Epoch 5/100
32/32 [==============================] - 2s 77ms/step - loss: 1.0152 - acc: 0.3438 - val_loss: 1.4512 - val_acc: 0.0000e+00
Epoch 6/100
32/32 [==============================] - 2s 75ms/step - loss: 0.9526 - acc: 0.4688 - val_loss: 1.3084 - val_acc: 0.3750
Epoch 7/100
 6/32 [====>.........................] - ETA: 1s - loss: 0.8493 - acc: 0.6667

32/32 [==============================] - 2s 76ms/step - loss: 1.4462 - acc: 0.3438 - val_loss: 1.3801 - val_acc: 0.2500
Epoch 8/100
32/32 [==============================] - 2s 77ms/step - loss: 1.2453 - acc: 0.3438 - val_loss: 1.3030 - val_acc: 0.2500
Epoch 9/100
32/32 [==============================] - 2s 76ms/step - loss: 1.2084 - acc: 0.3750 - val_loss: 1.2681 - val_acc: 0.2500
Epoch 10/100
32/32 [==============================] - 2s 77ms/step - loss: 1.0846 - acc: 0.3750 - val_loss: 1.3686 - val_acc: 0.1250
Epoch 11/100
32/32 [==============================] - 2s 78ms/step - loss: 1.6420 - acc: 0.3750 - val_loss: 1.5620 - val_acc: 0.0000e+00
Epoch 12/100
32/32 [==============================] - 2s 77ms/step - loss: 1.3805 - acc: 0.3125 - val_loss: 1.3416 - val_acc: 0.2500
Epoch 13/100
21/32 [==================>...........] - ETA: 0s - loss: 1.4472 - acc: 0.1905

32/32 [==============================] - 3s 78ms/step - loss: 1.4059 - acc: 0.2188 - val_loss: 1.5587 - val_acc: 0.3750
Epoch 14/100
32/32 [==============================] - 2s 77ms/step - loss: 1.5703 - acc: 0.1875 - val_loss: 1.5598 - val_acc: 0.0000e+00
Epoch 15/100
32/32 [==============================] - 3s 79ms/step - loss: 1.4031 - acc: 0.3438 - val_loss: 1.3707 - val_acc: 0.1250
Epoch 16/100
32/32 [==============================] - 3s 79ms/step - loss: 1.2462 - acc: 0.4062 - val_loss: 1.2618 - val_acc: 0.3750
Epoch 17/100
32/32 [==============================] - 2s 78ms/step - loss: 1.1404 - acc: 0.4688 - val_loss: 1.3237 - val_acc: 0.3750
Epoch 18/100
32/32 [==============================] - 2s 78ms/step - loss: 1.1883 - acc: 0.4062 - val_loss: 1.3023 - val_acc: 0.1250
Epoch 19/100
23/32 [====================>.........] - ETA: 0s - loss: 1.4049 - acc: 0.3478

32/32 [==============================] - 3s 81ms/step - loss: 1.2136 - acc: 0.4375 - val_loss: 1.2801 - val_acc: 0.3750
Epoch 20/100
32/32 [==============================] - 3s 79ms/step - loss: 1.1271 - acc: 0.4062 - val_loss: 1.2890 - val_acc: 0.1250
Epoch 21/100
32/32 [==============================] - 3s 79ms/step - loss: 1.0790 - acc: 0.5000 - val_loss: 1.5297 - val_acc: 0.1250
Epoch 22/100
32/32 [==============================] - 3s 79ms/step - loss: 1.0874 - acc: 0.4688 - val_loss: 1.2822 - val_acc: 0.2500
Epoch 23/100
32/32 [==============================] - 3s 79ms/step - loss: 0.9841 - acc: 0.4688 - val_loss: 1.7280 - val_acc: 0.3750
Epoch 24/100
32/32 [==============================] - 3s 79ms/step - loss: 1.2943 - acc: 0.4688 - val_loss: 1.5243 - val_acc: 0.2500
Epoch 25/100
25/32 [======================>.......] - ETA: 0s - loss: 1.2838 - acc: 0.4400

32/32 [==============================] - 3s 80ms/step - loss: 1.2615 - acc: 0.4375 - val_loss: 1.1514 - val_acc: 0.2500
Epoch 26/100
32/32 [==============================] - 3s 79ms/step - loss: 1.3032 - acc: 0.3438 - val_loss: 1.3718 - val_acc: 0.2500
Epoch 27/100
32/32 [==============================] - 2s 78ms/step - loss: 1.1456 - acc: 0.4688 - val_loss: 1.2865 - val_acc: 0.1250
Epoch 28/100
32/32 [==============================] - 3s 79ms/step - loss: 1.0715 - acc: 0.4688 - val_loss: 1.1075 - val_acc: 0.2500
Epoch 29/100
32/32 [==============================] - 3s 80ms/step - loss: 0.9556 - acc: 0.5312 - val_loss: 1.0724 - val_acc: 0.2500
Epoch 30/100
32/32 [==============================] - 3s 79ms/step - loss: 0.9082 - acc: 0.5625 - val_loss: 0.9565 - val_acc: 0.2500
Epoch 31/100
26/32 [=======================>......] - ETA: 0s - loss: 1.2913 - acc: 0.5000

...

32/32 [==============================] - 2s 77ms/step - loss: 1.3989 - acc: 0.0312 - val_loss: 1.3869 - val_acc: 0.2500
Epoch 74/100
32/32 [==============================] - 2s 77ms/step - loss: 1.3985 - acc: 0.0312 - val_loss: 1.3869 - val_acc: 0.2500
Epoch 75/100
32/32 [==============================] - 2s 77ms/step - loss: 1.3982 - acc: 0.0312 - val_loss: 1.3869 - val_acc: 0.2500
Epoch 76/100
32/32 [==============================] - 2s 77ms/step - loss: 1.3979 - acc: 0.0312 - val_loss: 1.3869 - val_acc: 0.2500
Epoch 77/100
32/32 [==============================] - 2s 78ms/step - loss: 1.3976 - acc: 0.0312 - val_loss: 1.3869 - val_acc: 0.2500
Epoch 78/100
32/32 [==============================] - 3s 80ms/step - loss: 1.3973 - acc: 0.0312 - val_loss: 1.3870 - val_acc: 0.2500
Epoch 79/100
24/32 [=====================>........] - ETA: 0s - loss: 1.4003 - acc: 0.0000e+00

32/32 [==============================] - 3s 79ms/step - loss: 1.3971 - acc: 0.0312 - val_loss: 1.3870 - val_acc: 0.2500
Epoch 80/100
32/32 [==============================] - 2s 78ms/step - loss: 1.3968 - acc: 0.0312 - val_loss: 1.3870 - val_acc: 0.2500
Epoch 81/100
32/32 [==============================] - 2s 77ms/step - loss: 1.3965 - acc: 0.0312 - val_loss: 1.3870 - val_acc: 0.2500
Epoch 82/100
32/32 [==============================] - 2s 77ms/step - loss: 1.3963 - acc: 0.0312 - val_loss: 1.3870 - val_acc: 0.2500
Epoch 83/100
32/32 [==============================] - 2s 78ms/step - loss: 1.3960 - acc: 0.0312 - val_loss: 1.3870 - val_acc: 0.2500
Epoch 84/100
32/32 [==============================] - 2s 78ms/step - loss: 1.3958 - acc: 0.0312 - val_loss: 1.3871 - val_acc: 0.2500
Epoch 85/100
22/32 [===================>..........] - ETA: 0s - loss: 1.3975 - acc: 0.0000e+00

32/32 [==============================] - 2s 77ms/step - loss: 1.3955 - acc: 0.0312 - val_loss: 1.3871 - val_acc: 0.2500
Epoch 86/100
32/32 [==============================] - 3s 78ms/step - loss: 1.3953 - acc: 0.0312 - val_loss: 1.3871 - val_acc: 0.2500
Epoch 87/100
32/32 [==============================] - 3s 82ms/step - loss: 1.3950 - acc: 0.0312 - val_loss: 1.3872 - val_acc: 0.2500
Epoch 88/100
32/32 [==============================] - 2s 78ms/step - loss: 1.3947 - acc: 0.0312 - val_loss: 1.3872 - val_acc: 0.2500
Epoch 89/100
32/32 [==============================] - 2s 77ms/step - loss: 1.3944 - acc: 0.0938 - val_loss: 1.3872 - val_acc: 0.2500
Epoch 90/100
32/32 [==============================] - 3s 78ms/step - loss: 1.3940 - acc: 0.0938 - val_loss: 1.3873 - val_acc: 0.2500
Epoch 91/100
24/32 [=====================>........] - ETA: 0s - loss: 1.3968 - acc: 0.0417

32/32 [==============================] - 3s 78ms/step - loss: 1.3935 - acc: 0.0938 - val_loss: 1.3874 - val_acc: 0.2500
Epoch 92/100
32/32 [==============================] - 3s 78ms/step - loss: 1.3928 - acc: 0.1250 - val_loss: 1.3876 - val_acc: 0.2500
Epoch 93/100
32/32 [==============================] - 2s 78ms/step - loss: 1.3952 - acc: 0.1250 - val_loss: 1.3873 - val_acc: 0.2500
Epoch 94/100
32/32 [==============================] - 2s 78ms/step - loss: 1.3940 - acc: 0.1562 - val_loss: 1.3874 - val_acc: 0.2500
Epoch 95/100
32/32 [==============================] - 2s 78ms/step - loss: 1.3917 - acc: 0.1562 - val_loss: 1.3880 - val_acc: 0.2500
Epoch 96/100
32/32 [==============================] - 3s 79ms/step - loss: 1.3958 - acc: 0.1250 - val_loss: 1.3875 - val_acc: 0.2500
Epoch 97/100
25/32 [======================>.......] - ETA: 0s - loss: 1.3943 - acc: 0.0400

32/32 [==============================] - 2s 78ms/step - loss: 1.3905 - acc: 0.1250 - val_loss: 1.3881 - val_acc: 0.2500
Epoch 98/100
32/32 [==============================] - 2s 78ms/step - loss: 1.3919 - acc: 0.1562 - val_loss: 1.3888 - val_acc: 0.2500
Epoch 99/100
32/32 [==============================] - 2s 76ms/step - loss: 1.3944 - acc: 0.1250 - val_loss: 1.3879 - val_acc: 0.2500
Epoch 100/100
32/32 [==============================] - 2s 77ms/step - loss: 1.3891 - acc: 0.1875 - val_loss: 1.3889 - val_acc: 0.2500
batch size : 1 -> 251 seconds for training with GPU


In [19]:
y_train_ = np.zeros((32,303,4))
y_train_[:,0,:] = y_train

y_test_ = np.zeros((8,303,4))
y_test_[:,0,:] = y_test

shuffle_data[:32,:,:6] = x_train
shuffle_data[:32,:,6:] = y_train_

shuffle_data[32:,:,:6] = x_test
shuffle_data[32:,:,6:] = y_test_


while True:
  np.random.shuffle(shuffle_data)

  x_train = shuffle_data[:32,:,:6]
  x_test = shuffle_data[32:,:,:6]
  y_train = shuffle_data[:32,0,6:].reshape(32,4)
  y_test = shuffle_data[32:,0,6:].reshape(8,4)
  if np.sum(y_test[:,0]) == 2:
      if np.sum(y_test[:,1]) == 2:
          if np.sum(y_test[:,2]) == 2:
              if np.sum(y_test[:,3]) == 2:
                  break


batch_size = 8
timesteps = SEQ_LENGTH
data_dim = 6

model = Sequential()
model.add(CuDNNLSTM(128, return_sequences=True, 
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(CuDNNLSTM(64))
model.add(Dense(32))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


st = time.time()
history = model.fit(x_train, y_train,
          batch_size=batch_size, epochs=100, shuffle=False,
          validation_data=(x_test, y_test),verbose=1)
ft = time.time()

print("batch size : %d -> %d seconds for training with CPU"%(batch_size,ft-st))


Train on 32 samples, validate on 8 samples
Epoch 1/100
32/32 [==============================] - 1s 35ms/step - loss: 1.4376 - acc: 0.0938 - val_loss: 1.4336 - val_acc: 0.0000e+00
Epoch 2/100
32/32 [==============================] - 0s 15ms/step - loss: 1.3810 - acc: 0.3125 - val_loss: 1.4326 - val_acc: 0.0000e+00
Epoch 3/100
32/32 [==============================] - 0s 13ms/step - loss: 1.3540 - acc: 0.4062 - val_loss: 1.4155 - val_acc: 0.1250
Epoch 4/100
32/32 [==============================] - 0s 12ms/step - loss: 1.3227 - acc: 0.3438 - val_loss: 1.3817 - val_acc: 0.1250
Epoch 5/100
32/32 [==============================] - 0s 11ms/step - loss: 1.2591 - acc: 0.3750 - val_loss: 1.3227 - val_acc: 0.2500
Epoch 6/100
32/32 [==============================] - 0s 11ms/step - loss: 1.1022 - acc: 0.5000 - val_loss: 0.9888 - val_acc: 0.3750
Epoch 7/100
32/32 [==============================] - 0s 11ms/step - loss: 0.9167 - acc: 0.5312 - val_loss: 0.9442 - val_acc: 0.3750
Epoch 8/100
32/32 [======

32/32 [==============================] - 0s 11ms/step - loss: 0.7753 - acc: 0.7188 - val_loss: 1.0247 - val_acc: 0.5000
Epoch 18/100
32/32 [==============================] - 0s 11ms/step - loss: 0.6932 - acc: 0.7500 - val_loss: 0.9965 - val_acc: 0.5000
Epoch 19/100
32/32 [==============================] - 0s 11ms/step - loss: 0.6751 - acc: 0.6875 - val_loss: 1.0025 - val_acc: 0.5000
Epoch 20/100
32/32 [==============================] - 0s 11ms/step - loss: 0.6427 - acc: 0.6562 - val_loss: 1.0330 - val_acc: 0.5000
Epoch 21/100
32/32 [==============================] - 0s 11ms/step - loss: 0.5797 - acc: 0.7500 - val_loss: 0.9846 - val_acc: 0.5000
Epoch 22/100
32/32 [==============================] - 0s 11ms/step - loss: 0.5146 - acc: 0.8125 - val_loss: 0.9324 - val_acc: 0.5000
Epoch 23/100
32/32 [==============================] - 0s 11ms/step - loss: 0.5044 - acc: 0.7812 - val_loss: 0.8724 - val_acc: 0.5000
Epoch 24/100
32/32 [==============================] - 0s 11ms/step - loss: 0.4559 

32/32 [==============================] - 0s 11ms/step - loss: 0.4237 - acc: 0.7500 - val_loss: 0.8394 - val_acc: 0.6250
Epoch 34/100
32/32 [==============================] - 0s 11ms/step - loss: 0.3631 - acc: 0.8750 - val_loss: 1.1322 - val_acc: 0.5000
Epoch 35/100
32/32 [==============================] - 0s 11ms/step - loss: 0.3750 - acc: 0.8438 - val_loss: 1.2769 - val_acc: 0.5000
Epoch 36/100
32/32 [==============================] - 0s 11ms/step - loss: 0.3418 - acc: 0.8438 - val_loss: 0.8781 - val_acc: 0.6250
Epoch 37/100
32/32 [==============================] - 0s 11ms/step - loss: 0.3134 - acc: 0.8750 - val_loss: 1.6383 - val_acc: 0.5000
Epoch 38/100
32/32 [==============================] - 0s 11ms/step - loss: 0.4014 - acc: 0.8125 - val_loss: 1.7264 - val_acc: 0.5000
Epoch 39/100
32/32 [==============================] - 0s 11ms/step - loss: 0.3127 - acc: 0.8438 - val_loss: 1.2229 - val_acc: 0.5000
Epoch 40/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2960 

32/32 [==============================] - 0s 11ms/step - loss: 0.2272 - acc: 0.8750 - val_loss: 1.2455 - val_acc: 0.6250
Epoch 51/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2092 - acc: 0.8750 - val_loss: 1.2801 - val_acc: 0.7500
Epoch 52/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2043 - acc: 0.9062 - val_loss: 1.2728 - val_acc: 0.8750
Epoch 53/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2086 - acc: 0.9062 - val_loss: 1.2403 - val_acc: 0.8750
Epoch 54/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2056 - acc: 0.8750 - val_loss: 1.4477 - val_acc: 0.6250
Epoch 55/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2887 - acc: 0.8750 - val_loss: 1.0200 - val_acc: 0.6250
Epoch 56/100
32/32 [==============================] - 0s 11ms/step - loss: 0.3289 - acc: 0.8438 - val_loss: 1.0063 - val_acc: 0.5000
Epoch 57/100
32/32 [==============================] - 0s 11ms/step - loss: 0.5697 

32/32 [==============================] - 0s 11ms/step - loss: 0.2415 - acc: 0.9375 - val_loss: 1.1598 - val_acc: 0.7500
Epoch 68/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2196 - acc: 0.9375 - val_loss: 1.1601 - val_acc: 0.7500
Epoch 69/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2047 - acc: 0.9062 - val_loss: 1.1596 - val_acc: 0.7500
Epoch 70/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1983 - acc: 0.9375 - val_loss: 1.1553 - val_acc: 0.7500
Epoch 71/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1786 - acc: 0.9375 - val_loss: 1.1648 - val_acc: 0.7500
Epoch 72/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2647 - acc: 0.8438 - val_loss: 1.2557 - val_acc: 0.6250
Epoch 73/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2016 - acc: 0.8750 - val_loss: 1.2453 - val_acc: 0.5000
Epoch 74/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1833 

32/32 [==============================] - 0s 11ms/step - loss: 0.3123 - acc: 0.8438 - val_loss: 1.6425 - val_acc: 0.5000
Epoch 85/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2253 - acc: 0.8750 - val_loss: 1.7066 - val_acc: 0.5000
Epoch 86/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2364 - acc: 0.8750 - val_loss: 1.5735 - val_acc: 0.5000
Epoch 87/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2115 - acc: 0.8750 - val_loss: 1.4246 - val_acc: 0.5000
Epoch 88/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2043 - acc: 0.9062 - val_loss: 1.3632 - val_acc: 0.6250
Epoch 89/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2021 - acc: 0.9688 - val_loss: 1.4153 - val_acc: 0.6250
Epoch 90/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1902 - acc: 0.9688 - val_loss: 1.4586 - val_acc: 0.6250
Epoch 91/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1836 